# bet365-scraper

In this script, we scrap football information from bet365

In [2]:
### imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [42]:
### initialize driver and connect to website
driver = webdriver.Chrome()
driver.get('https://www.bet365.com/en/')
# go to sports page
# click on sports banner, once it is loaded
sports_banner = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.ID, 'dv1'))
)
sports_banner.click()
# change odds to decimal
oddsHeaderModule = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "hm-HeaderModule"))
)
oddsSelectorParent = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "hm-HeaderModule_Secondary"))
)
oddsSelector = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "hm-OddsDropDownSelections"))
)
oddsSelector.click()
oddsSelector.find_elements(By.CLASS_NAME, "hm-DropDownSelections_Item")[1].click()
# select Soccer
for element in driver.find_elements(By.CLASS_NAME, "wn-Classification"):
    if element.text == "Soccer": 
        element.click()
        break
### select Italy A B games
fullTimeResults = WebDriverWait(driver,10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "sm-MarketGroup"))
)[0]
for element in WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "sm-Market"))):
    if element.text == 'Italy':
        element.click()
        break
WebDriverWait(element,10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "sm-Market_HeaderCheckBox"))
).click()
el = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "sm-CreateCouponTab_Button"))
)
el.click()

WebDriverException: Message: unknown error: Element is not clickable at point (599, -5)
  (Session info: chrome=64.0.3282.140)
  (Driver info: chromedriver=2.34.522932 (4140ab217e1ca1bec0c4b4d1b148f3361eb3a03e),platform=Mac OS X 10.12.6 x86_64)


In [80]:
### connect to bet365.com and travel to games list
# connect to site
url = 'https://www.bet365.com/en/'
driver = webdriver.Firefox()
driver.get(url)

# click on sports banner, once it is loaded
sports_banner = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.ID, 'dv1'))
)
sports_banner.click()

# click on 'Elite Euro List', once previous page is loaded
top_coupons = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'tc-TopCouponStemRenderer'))
)
top_coupons.find_elements_by_class_name('tc-TopCouponLinkButton')[1].click()

# change odds to decimal
odds_selector = WebDriverWait(driver,10).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'hm-OddsDropDownSelections.hm-DropDownSelections'))
)
odds_selector.click()

odds_items = WebDriverWait(driver,10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, 'hm-DropDownSelections_Item'))
)
odds_items[1].click()

TimeoutException: Message: 


In [52]:
### get list of the major leagues matches
leagues = driver.find_element(By.CLASS_NAME, 'gl-MarketGrid').find_elements(By.CLASS_NAME, 'gl-MarketGroup.cm-CouponMarketGroup')

In [53]:
### extract information from each league
league = leagues[0]
# get title of the league
league_title = str(league.find_element(By.CLASS_NAME, 'cm-CouponMarketGroupButton_Title').text)
# get grid, containing matches, times and odds
matches_odds_grid = league.find_element(By.CLASS_NAME, 'gl-MarketGroup_Wrapper')

In [54]:
# extract matches and odds from grid
matches_row = matches_odds_grid.find_element(By.CLASS_NAME, 'sl-MarketCouponFixtureLabelBase.gl-Market_HasLabels')
[odds_1_row, odds_X_row, odds_2_row] = matches_odds_grid.find_elements(By.CLASS_NAME, 'sl-MarketCouponValuesExplicit33')

In [55]:
# get only elements containing date or game
elements_to_filter = matches_row.find_elements(By.CSS_SELECTOR, 'div')
first_date_class_name = u'gl-MarketColumnHeader sl-MarketHeaderLabel sl-MarketHeaderLabel_Date '
another_date_class_name = u'gl-MarketColumnHeader sl-MarketHeaderLabel_Date sl-MarketHeaderLabel '
game_class_name = u'sl-CouponParticipantWithBookCloses sl-CouponParticipantIPPGBase '
last_game_class_name = u'sl-CouponParticipantWithBookCloses sl-CouponParticipantIPPGBase sl-MarketCouponAdvancedBase_LastChild '
classes_to_keep = [first_date_class_name, another_date_class_name, game_class_name, last_game_class_name ]
filtered_elements = [element for element in elements_to_filter if element.get_attribute('class') in classes_to_keep]

In [75]:
# split filtered elements into blocks, each one for a different day
date_indeces = [i for i, j in enumerate(filtered_elements) 
                if j.get_attribute('class') in [first_date_class_name, another_date_class_name]]

# get list of elements, frow which slice by date
games_list = filtered_elements
odds_1_list = odds_1_row.text.split('\n')
odds_X_list = odds_X_row.text.split('\n')
odds_2_list = odds_2_row.text.split('\n')
# define empty dictionaries, to contain games and odds for each date
block_of_games = []
block_of_1_odds = []
block_of_X_odds = []
block_of_2_odds = []
# for each date (starting from last), get slice of games and odds
for date in date_indeces[::-1]:
    # append block for the given date
    block_of_games.append(games_list[date:])
    block_of_1_odds.append(odds_1_list[date:])
    block_of_X_odds.append(odds_X_list[date:])
    block_of_2_odds.append(odds_2_list[date:])
    # remove entries for that date from original data
    games_list = games_list[:date]
    odds_1_list = odds_1_list[:date]
    odds_X_list = odds_X_list[:date]
    odds_2_list = odds_2_list[:date]
# finally, revert blocks, in order the have them in cronological order
block_of_games = block_of_games[::-1]
block_of_1_odds = block_of_1_odds[::-1]
block_of_X_odds = block_of_X_odds[::-1]
block_of_2_odds = block_of_2_odds[::-1]

In [76]:
block_of_1_odds

[[u'1',
  u'6.50',
  u'1.50',
  u'7.00',
  u'2.15',
  u'1.85',
  u'2.05',
  u'2.05',
  u'1.12'],
 [u'1', u'5.75'],
 [u'1', u'12.00'],
 [u'1', u'6.00', u'2.37', u'9.00'],
 [u'1', u'1.22', u'2.30', u'2.25', u'1.66', u'1.10', u'2.25', u'2.15']]

In [62]:
filtered_elements

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="3d09f19e-4165-f94c-97df-d2aed36fa7dc", element="aeb61d24-f2c6-1a47-a5c7-aa30f3639bbf")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="3d09f19e-4165-f94c-97df-d2aed36fa7dc", element="d842fc80-e4e5-ed40-9b12-fe11cc556a68")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="3d09f19e-4165-f94c-97df-d2aed36fa7dc", element="15166e86-69b2-c043-bb72-05bf86f11e9f")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="3d09f19e-4165-f94c-97df-d2aed36fa7dc", element="1f84a0a0-27fd-b949-9a2e-f1eded70d964")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="3d09f19e-4165-f94c-97df-d2aed36fa7dc", element="495f7fc5-0511-fc41-b251-a0f1452d1f6c")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="3d09f19e-4165-f94c-97df-d2aed36fa7dc", element="73638dca-8b6a-784f-8398-d68f88a6453b")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement 

In [40]:
tst.remove(tst[-2:])

ValueError: list.remove(x): x not in list